In [1]:
import tweepy
API_KEY = '8xvUEF0lAb858eEnGWu0eYH5j'
API_SECRET_KEY = 'sKb1mc2aLkdb2QxFpEXU6VHWiUfOCOUKy9BMJSzkF2abMpjQP3'
ACCESS_TOEKN = '1317910496142569475-CLZoQdkMmOMpH6Xzv3B3SDIxOii8bz'
ACCESS_TOKEN_SECRET = 'xFscaufPjGdMG7Tq555Kp6HwoFDCaZHXJnNWcwKbpDWmf'
auth  = tweepy.OAuthHandler(API_KEY, \
                            API_SECRET_KEY)
auth.set_access_token(ACCESS_TOEKN,  \
                      ACCESS_TOKEN_SECRET)
api = tweepy.API(auth)

In [2]:
TRACK_WORDS = ['depression', 'games']
TABLE_NAME = "Mental_Health"
TABLE_ATTRIBUTES = "id_str VARCHAR(255), created_at DATETIME, text VARCHAR(255), \
            depression VARCHAR(255), user_created_at VARCHAR(255), user_location VARCHAR(255), \
            user_description VARCHAR(255), user_followers_count INT, longitude DOUBLE, latitude DOUBLE, \
            retweet_count INT, favorite_count INT"

In [13]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

FullData = pd.read_csv(r'C:\Users\eagle\Desktop\trainingdata\CombinedTweetsTraining.csv')

TrainingData, ValidtingData = train_test_split(FullData, test_size=0.3)

TestingData = pd.read_csv(r'C:\Users\eagle\Desktop\trainingdata\test1000.csv')

In [14]:
import nltk
import re, string
from nltk.tag import pos_tag
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import sent_tokenize, word_tokenize
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
from tqdm import tqdm
from nltk.corpus import stopwords
stopwordEn = stopwords.words('english')
from nltk.corpus import wordnet
nltk.download('wordnet')
import re, string, unicodedata
import nltk
from nltk import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from nltk.stem.porter import PorterStemmer

def tokenization(word):
  word = nltk.word_tokenize(word)
  return word


def lemmatize_sentence_And_POStag(tokens):
    lemmatizer = WordNetLemmatizer()
    porter = PorterStemmer()
    stem_tokens = porter.stem(tokens) 
    lemmatized_sentence = []
    for word, tag in pos_tag(tokens):
        if tag.startswith('NN'):
            pos = 'n'
        elif tag.startswith('VB'):
            pos = 'v'
        elif tag.startswith('ADJ'):
            pos = 'j'
        else:
            pos = 'a'
        lemmatized_sentence.append(lemmatizer.lemmatize(word, pos))
    return lemmatized_sentence


def remove_non_ascii(words):
    """Remove non-ASCII characters from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = unicodedata.normalize('NFKD', word).encode('ascii', 'ignore').decode('utf-8', 'ignore')
        new_words.append(new_word)
    return new_words

def to_lowercase(words):
    """Convert all characters to lowercase from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = word.lower()
        new_words.append(new_word)
    return new_words

def remove_punctuation(words):
    """Remove punctuation from list of tokenized words"""
    new_words = []
    for word in words:
        new_word1 = re.sub(r'[^\w\s]', '', word)
        new_word = re.sub(r'(https|http)?:\/\/(\w|\.|\/|\?|\=|\&|\%)*\b|@\w+|#', '', new_word1, flags=re.MULTILINE)
        if new_word != '':
            new_words.append(new_word)
    return new_words

def remove_stopwords(words):
    """Remove stop words from list of tokenized words"""
    new_words = []
    for word in words:
        if word not in stopwords.words('english'):
            new_words.append(word)
    return new_words

def normalize(words):
    words = tokenization(words)
    words = remove_non_ascii(words)
    words = to_lowercase(words)
    words = remove_punctuation(words)
    words = remove_stopwords(words)
    return words

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\eagle\anaconda3\lib\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\eagle\anaconda3\lib\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\eagle\anaconda3\lib\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\eagle\anaconda3\lib\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [15]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import SGDClassifier

text_clf = Pipeline([
               
    ('vect', CountVectorizer(analyzer=normalize and lemmatize_sentence_And_POStag)), # data preproccessing  


    
    ('clf', SGDClassifier()) # suport vector machine model

])

In [24]:
text_clf.fit(FullData.Text, FullData.Sentiment)

Pipeline(steps=[('vect',
                 CountVectorizer(analyzer=<function lemmatize_sentence_And_POStag at 0x0000017577CC4790>)),
                ('clf', SGDClassifier())])

In [15]:
from joblib import dump, load
dump(text_clf, 'depression_model.joblib')

['depression_model.joblib']

In [25]:
predicted = text_clf.predict(TestingData.text)
print(predicted)

['negative' 'positive' 'positive' ... 'negative' 'negative' 'negative']


In [26]:
from sklearn import metrics

print("Accuracy:", metrics.accuracy_score(TestingData.Sentiment, predicted)*100) # give the accuracy in percentage 

target_names = ['negative', 'positive'] # create a confusion matrix table with negative and positive only

print(metrics.classification_report(TestingData.Sentiment, predicted, target_names=target_names)) # comparing the real sentimet values with the predicted values 

# confusion matrix
pd.DataFrame(metrics.confusion_matrix(TestingData.Sentiment, predicted),
             columns=target_names,index=target_names)


Accuracy: 75.93152064451158
              precision    recall  f1-score   support

    negative       0.86      0.85      0.86      1661
    positive       0.27      0.27      0.27       325

    accuracy                           0.76      1986
   macro avg       0.56      0.56      0.56      1986
weighted avg       0.76      0.76      0.76      1986



,negative,positive
negative,1420,241
positive,237,88
